In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
firstnames = pd.read_excel('../data/names_freq.xlsx',sheet_name='Given names',usecols="A:I")

In [3]:
lastnames = pd.read_excel('../data/names_freq.xlsx',sheet_name='Family names',usecols="A:H")

checks

In [7]:
lastnames.loc[lastnames.name == 'TSETHLIKAI',]

,name,count,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
116909,TSETHLIKAI,149,0.0,0.0,0.0,0.9664,0.0,0.0


In [8]:
racegrps = ['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']

In [9]:
(lastnames[racegrps].sum(axis=1)>1.01).sum()
(lastnames[racegrps].sum(axis=1)<0.9).sum()
lastnames[lastnames[racegrps].sum(axis=1)<0.9]

,name,count,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
81314,CHACO,233,0.2318,0.0000,0.3476,0.1288,0.0000,0.1760
83871,BONGA,224,0.4598,0.1563,0.1384,0.1429,0.0000,0.0000
89941,ANE,206,0.2816,0.0000,0.1942,0.0000,0.1553,0.2476
93162,DANNY,197,0.5482,0.0964,0.1320,0.1117,0.0000,0.0000
100327,BRIJMOHAN,180,0.1333,0.1722,0.5111,0.0000,0.0833,0.0000
...,...,...,...,...,...,...,...,...
161081,SLAKEY,100,0.8900,0.0000,0.0000,0.0000,0.0000,0.0000
161281,UPWARD,100,0.8900,0.0000,0.0000,0.0000,0.0000,0.0000
161695,EIDINGER,100,0.8900,0.0000,0.0000,0.0000,0.0000,0.0000
162036,GOZE,100,0.4800,0.0000,0.4100,0.0000,0.0000,0.0000


There are some cases that sum less than 1. This is for privacy preservation. I will normalize so everything sums up to 1

In [10]:
# lastnames[racegrps] = lastnames[racegrps]/lastnames[racegrps].sum(axis=1)
lastnames.loc[:,racegrps] = lastnames.loc[:,racegrps].div(lastnames[racegrps].sum(axis=1), axis=0)

In [12]:
(firstnames[racegrps].sum(axis=1)>101).sum()
(firstnames[racegrps].sum(axis=1)<99).sum()

0

No problems with first names, but normalize probs to 0-1

In [13]:
firstnames.loc[:,racegrps] = firstnames.loc[:,racegrps].div(firstnames[racegrps].sum(axis=1), axis=0)

Check: Which categories to use

In [14]:
lastnames[['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']].multiply(lastnames["count"], axis="index").sum()/lastnames["count"].sum()


pctwhite       0.644342
pctblack       0.119751
pctapi         0.048627
pctaian        0.006942
pct2prace      0.017628
pcthispanic    0.162711
dtype: float64

I'll keep `'pctwhite','pctblack','pctapi','pcthispanic'` 

In [15]:
# lastnames['pctother'] = lastnames['pctaian'] + lastnames['pct2prace']
# firstnames['pctother'] = firstnames['pctaian'] + firstnames['pct2prace']

remove pctaian and pct2prace and renormalize

In [16]:
lastnames= lastnames.loc[:,['name','count','pctwhite','pcthispanic','pctblack','pctapi']]
lastnames[['pctwhite','pcthispanic','pctblack','pctapi']] = lastnames[['pctwhite','pcthispanic','pctblack','pctapi']].div(
    lastnames[['pctwhite','pcthispanic','pctblack','pctapi']].sum(axis=1), axis=0)

firstnames= firstnames.loc[:,['firstname','obs','pctwhite','pcthispanic','pctblack','pctapi']]
firstnames[['pctwhite','pcthispanic','pctblack','pctapi']] = firstnames[['pctwhite','pcthispanic','pctblack','pctapi']].div(
    firstnames[['pctwhite','pcthispanic','pctblack','pctapi']].sum(axis=1), axis=0)

In [17]:
lastnames  = lastnames.rename({'name':'lastname'},axis=1).set_index('lastname')[['count','pctwhite','pcthispanic','pctblack','pctapi']]
firstnames = firstnames.rename({'obs':'count'},axis=1).set_index('firstname')[['count','pctwhite','pcthispanic','pctblack','pctapi']]

In [ ]:
lastnames

In [29]:
#there are some names that only had values for pctaian and pct2prace. I set them to 0
lastnames.loc[lastnames.isna()['pctwhite'].values,]

,count,pctwhite,pcthispanic,pctblack,pctapi
lastname,,,,,
HOSTEEN,313,NaN,NaN,NaN,NaN
YELLOWMAN,310,NaN,NaN,NaN,NaN
BITSUI,285,NaN,NaN,NaN,NaN
HENIO,270,NaN,NaN,NaN,NaN
DECLAY,267,NaN,NaN,NaN,NaN
...,...,...,...,...,...
NOCKIDENEH,101,NaN,NaN,NaN,NaN
DENETSOSIE,100,NaN,NaN,NaN,NaN
HATATHLIE,100,NaN,NaN,NaN,NaN


In [34]:
firstnames.loc[firstnames.isna()['pctwhite'].values,]

,count,pctwhite,pcthispanic,pctblack,pctapi
firstname,,,,,


In [32]:
lastnames = lastnames.fillna(0)
lastnames.loc[lastnames.isna()['pctwhite'].values,]

,count,pctwhite,pcthispanic,pctblack,pctapi
lastname,,,,,


In [35]:
lastnames.loc['TSETHLIKAI',]

count          149.0
pctwhite         0.0
pcthispanic      0.0
pctblack         0.0
pctapi           0.0
Name: TSETHLIKAI, dtype: float64

# First name normalization

In [36]:
#     lastnames_prop = lastnames[['pctwhite','pcthispanic','pctblack','pctapi','pctother']].multiply(lastnames["count"], axis="index").sum()/lastnames["count"].sum()
#     firstnames_prop = firstnames[['pctwhite','pcthispanic','pctblack','pctapi','pctother']].multiply(firstnames["count"], axis="index").sum()/firstnames["count"].sum()
#     ratio = firstnames_general_prop/lastnames_general_prop
#     normalized_names = firstnames[['pctwhite','pcthispanic','pctblack','pctapi','pctother']]/ratio
#     normalized_names = normalized_names.divide(normalized_names.sum(axis=1),axis='index')
#     normalized_names = normalized_names.join(firstnames['count'])

In [37]:
def first_name_normalization(lastnames,firstnames):
    
    lastnames_freq = lastnames[['pctwhite','pcthispanic','pctblack','pctapi']].multiply(lastnames["count"], axis="index")
    firstnames_freq = firstnames[['pctwhite','pcthispanic','pctblack','pctapi']].multiply(firstnames["count"], axis="index")
    
    lastnames_total = lastnames_freq.sum()
    firstnames_total = firstnames_freq.sum()

    expansion_factors = lastnames_total/firstnames_total
    expanded_names = firstnames_freq*expansion_factors
    expanded_names['count'] = expanded_names.sum(axis=1)
    expanded_names[['pctwhite','pcthispanic','pctblack','pctapi']] = expanded_names[['pctwhite','pcthispanic','pctblack','pctapi']].divide(expanded_names['count'], axis= 'index')
    return expanded_names

In [38]:
normalized_names = first_name_normalization(lastnames,firstnames)

In [39]:
lastnames[['pctwhite','pcthispanic','pctblack','pctapi']].multiply(lastnames["count"], axis="index").sum()/lastnames["count"].sum()

pctwhite       0.661099
pcthispanic    0.164788
pctblack       0.123754
pctapi         0.050301
dtype: float64

In [40]:
firstnames[['pctwhite','pcthispanic','pctblack','pctapi']].multiply(firstnames["count"], axis="index").sum()/firstnames["count"].sum()

pctwhite       0.825763
pcthispanic    0.068955
pctblack       0.042236
pctapi         0.063046
dtype: float64

In [41]:
normalized_names[['pctwhite','pcthispanic','pctblack','pctapi']].multiply(normalized_names["count"], axis="index").sum()/normalized_names["count"].sum()

pctwhite       0.661137
pcthispanic    0.164798
pctblack       0.123761
pctapi         0.050304
dtype: float64

In [42]:
normalized_names.to_csv('../data/names_normalized.csv')

## Save

In [43]:
lastnames

,count,pctwhite,pcthispanic,pctblack,pctapi
lastname,,,,,
ALL OTHER LAST NAMES,29312001,0.688391,0.141190,0.088102,0.082318
SMITH,2442977,0.731607,0.024765,0.238469,0.005159
JOHNSON,1932812,0.611088,0.024456,0.358860,0.005596
WILLIAMS,1625252,0.474684,0.025835,0.494708,0.004773
BROWN,1437026,0.600021,0.026092,0.368606,0.005281
...,...,...,...,...,...
BULAR,100,0.870000,0.000000,0.130000,0.000000
WIGGLETON,100,0.390000,0.000000,0.610000,0.000000
SNEARL,100,0.000000,0.000000,1.000000,0.000000


In [44]:
lastnames = lastnames.drop('count',axis=1)
firstnames = firstnames.drop('count',axis=1)
normalized_names = normalized_names.drop('count',axis=1)

In [45]:
lastnames_dict = dict(zip(lastnames.index,np.array(lastnames)))
firstnames_dict = dict(zip(firstnames.index,np.array(firstnames)))
firstnames_dict_normalized =  dict(zip(normalized_names.index,np.array(normalized_names)))

In [46]:
dictionaries = {'lastnames_dict':lastnames_dict,'firstnames_dict':firstnames_dict,'firstnames_norm_dict':firstnames_dict_normalized}

I save the dicts

In [47]:
pickle.dump(dictionaries, open('libs/names_prob.p', 'wb'))

Lib/RaceDistribution.py